In [1]:
#2020-12-13 cowrieログ分析について
import pandas as pd
import datetime
import gzip
fname = 'cowrie.json.2020-12-29'
access = pd.read_json(fname,lines=True)
df=pd.DataFrame(access)
df['timestamp']=pd.to_datetime(df['timestamp'])
df

,eventid,ttylog,size,shasum,duplicate,duration,message,sensor,timestamp,src_ip,...,langCS,username,password,data,id,outfile,destfile,filename,width,height
0,cowrie.log.closed,var/lib/cowrie/tty/64426356ffcabc3671e5bd0acff...,2.0,64426356ffcabc3671e5bd0acff75ec85278dc0d4ff5da...,1.0,0.520375,Closing TTY Log: var/lib/cowrie/tty/64426356ff...,ik1-448-56163,2020-12-29 00:00:00.179807+00:00,51.91.8.222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cowrie.session.params,NaN,NaN,NaN,NaN,NaN,[],ik1-448-56163,2020-12-29 00:00:00.466643+00:00,51.91.8.222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cowrie.command.input,NaN,NaN,NaN,NaN,NaN,CMD: top,ik1-448-56163,2020-12-29 00:00:00.467146+00:00,51.91.8.222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cowrie.session.connect,NaN,NaN,NaN,NaN,NaN,New connection: 5.188.86.178:61490 (133.125.60...,ik1-448-56163,2020-12-29 00:00:00.532028+00:00,5.188.86.178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cowrie.client.version,NaN,NaN,NaN,NaN,NaN,Remote SSH version: b'SSH-2.0-Go',ik1-448-56163,2020-12-29 00:00:00.532569+00:00,5.188.86.178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287849,cowrie.direct-tcpip.request,NaN,NaN,NaN,NaN,NaN,direct-tcp connection request to 212.159.9.200...,ik1-448-56163,2020-12-29 23:59:58.663555+00:00,5.188.86.206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287850,cowrie.login.failed,NaN,NaN,NaN,NaN,NaN,login attempt [albert/albert123] failed,ik1-448-56163,2020-12-29 23:59:58.719157+00:00,111.1.165.12,...,NaN,albert,albert123,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287851,cowrie.direct-tcpip.request,NaN,NaN,NaN,NaN,NaN,direct-tcp connection request to imap.gmx.net:...,ik1-448-56163,2020-12-29 23:59:59.541958+00:00,5.182.39.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287852,cowrie.direct-tcpip.data,NaN,NaN,NaN,NaN,NaN,discarded direct-tcp forward request 5 to imap...,ik1-448-56163,2020-12-29 23:59:59.796920+00:00,5.182.39.64,...,NaN,NaN,NaN,"b""\x16\x03\x03\x00\xc7\x01\x00\x00\xc3\x03\x03...",5.0,NaN,NaN,NaN,NaN,NaN


In [11]:
#ログインしようとしたID（ユーザーネーム）
df1=df.query('eventid == "cowrie.login.failed"')
print(df1['username'])
noid=df1['username'].value_counts()
noid1=df1['username'].value_counts(normalize=True)
print(noid)
print("パスワード頻度")
print(noid1)

52        esadmin
95           ftp1
119            sl
126           bso
198        marcos
           ...   
287606      nproc
287765       news
287779        lin
287789       test
287850     albert
Name: username, Length: 6988, dtype: object
nproc         1418
admin          721
test           156
ubuntu         131
user            94
              ... 
develop          1
zach             1
afa              1
qa               1
financeiro       1
Name: username, Length: 1477, dtype: int64
パスワード頻度
nproc         0.202919
admin         0.103177
test          0.022324
ubuntu        0.018746
user          0.013452
                ...   
develop       0.000143
zach          0.000143
afa           0.000143
qa            0.000143
financeiro    0.000143
Name: username, Length: 1477, dtype: float64


In [2]:
#実行に成功したコマンド
df1=df.query('eventid == "cowrie.command.input"')
print(df1['input'])

2                                                       top
8                                                     uname
14                                                 uname -a
17                                       lscpu | grep Model
21        cd ~ && rm -rf .ssh && mkdir .ssh && echo "ssh...
                                ...                        
287681    cd ~ && rm -rf .ssh && mkdir .ssh && echo "ssh...
287687                                                uname
287690                                             uname -a
287693                                   lscpu | grep Model
287696    cd ~ && rm -rf .ssh && mkdir .ssh && echo "ssh...
Name: input, Length: 21641, dtype: object


In [3]:
#実行に失敗したコマンド
df1=df.query('eventid == "cowrie.command.failed"')
print(df1['input'])

132211                 ./upnpsetup
158123    Enter new UNIX password:
202608    Enter new UNIX password:
206526    Enter new UNIX password:
207169    Enter new UNIX password:
208766    Enter new UNIX password:
213060    Enter new UNIX password:
228385    Enter new UNIX password:
Name: input, dtype: object


In [4]:
#ログインに失敗したログ
df2=df.query('eventid == "cowrie.login.failed"')
print("ログイン失敗回数："+str(len(df2)))
nopassword=df2['password'].value_counts()
nopassword1=df2['password'].value_counts(normalize=True)
print(nopassword)
print("パスワード頻度")
print(nopassword1)

ログイン失敗回数：6988
nproc            1418
123456            362
123               136
password          113
1234              101
                 ... 
we123               1
Sakura*123456       1
jenkins@123         1
sakura2008          1
                    1
Name: password, Length: 2374, dtype: int64
パスワード頻度
nproc            0.202919
123456           0.051803
123              0.019462
password         0.016171
1234             0.014453
                   ...   
we123            0.000143
Sakura*123456    0.000143
jenkins@123      0.000143
sakura2008       0.000143
                 0.000143
Name: password, Length: 2374, dtype: float64


In [5]:
#ログインに成功したログ
df2=df.query('eventid == "cowrie.login.success"')
print("ログイン成功回数："+str(len(df2)))
password=df2['password'].value_counts()
password1=df2['password'].value_counts(normalize=True)
print(password)
print("パスワード頻度")
print(password1)

ログイン成功回数：20830
admin         19212
1234567890       39
1234             32
malcolm          30
ethan            26
              ...  
abcd#1234         1
123456@a          1
admin_123         1
Test@12345        1
asdfghjkl         1
Name: password, Length: 836, dtype: int64
パスワード頻度
admin         0.922324
1234567890    0.001872
1234          0.001536
malcolm       0.001440
ethan         0.001248
                ...   
abcd#1234     0.000048
123456@a      0.000048
admin_123     0.000048
Test@12345    0.000048
asdfghjkl     0.000048
Name: password, Length: 836, dtype: float64


In [6]:
#ダウンロードに成功
df.query('eventid == "cowrie.session.file_download"')

,eventid,ttylog,size,shasum,duplicate,duration,message,sensor,timestamp,src_ip,...,langCS,username,password,data,id,outfile,destfile,filename,width,height
25,cowrie.session.file_download,NaN,NaN,a8460f446be540410004b1a8db4083773fa46f7fe76fa8...,1.0,NaN,Saved redir contents with SHA-256 a8460f446be5...,ik1-448-56163,2020-12-29 00:00:04.170455+00:00,51.91.8.222,...,NaN,NaN,NaN,NaN,NaN,var/lib/cowrie/downloads/a8460f446be540410004b...,/root/.ssh/authorized_keys,NaN,NaN,NaN
496,cowrie.session.file_download,NaN,NaN,a8460f446be540410004b1a8db4083773fa46f7fe76fa8...,1.0,NaN,Saved redir contents with SHA-256 a8460f446be5...,ik1-448-56163,2020-12-29 00:03:00.661418+00:00,51.91.8.222,...,NaN,NaN,NaN,NaN,NaN,var/lib/cowrie/downloads/a8460f446be540410004b...,/root/.ssh/authorized_keys,NaN,NaN,NaN
906,cowrie.session.file_download,NaN,NaN,a8460f446be540410004b1a8db4083773fa46f7fe76fa8...,1.0,NaN,Saved redir contents with SHA-256 a8460f446be5...,ik1-448-56163,2020-12-29 00:05:24.078095+00:00,112.196.52.58,...,NaN,NaN,NaN,NaN,NaN,var/lib/cowrie/downloads/a8460f446be540410004b...,/root/.ssh/authorized_keys,NaN,NaN,NaN
1002,cowrie.session.file_download,NaN,NaN,a8460f446be540410004b1a8db4083773fa46f7fe76fa8...,1.0,NaN,Saved redir contents with SHA-256 a8460f446be5...,ik1-448-56163,2020-12-29 00:05:43.351278+00:00,103.212.120.87,...,NaN,NaN,NaN,NaN,NaN,var/lib/cowrie/downloads/a8460f446be540410004b...,/root/.ssh/authorized_keys,NaN,NaN,NaN
1119,cowrie.session.file_download,NaN,NaN,a8460f446be540410004b1a8db4083773fa46f7fe76fa8...,1.0,NaN,Saved redir contents with SHA-256 a8460f446be5...,ik1-448-56163,2020-12-29 00:06:05.169419+00:00,51.91.8.222,...,NaN,NaN,NaN,NaN,NaN,var/lib/cowrie/downloads/a8460f446be540410004b...,/root/.ssh/authorized_keys,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286916,cowrie.session.file_download,NaN,NaN,a8460f446be540410004b1a8db4083773fa46f7fe76fa8...,1.0,NaN,Saved redir contents with SHA-256 a8460f446be5...,ik1-448-56163,2020-12-29 23:55:30.594713+00:00,164.132.24.255,...,NaN,NaN,NaN,NaN,NaN,var/lib/cowrie/downloads/a8460f446be540410004b...,/root/.ssh/authorized_keys,NaN,NaN,NaN
287174,cowrie.session.file_download,NaN,NaN,a8460f446be540410004b1a8db4083773fa46f7fe76fa8...,1.0,NaN,Saved redir contents with SHA-256 a8460f446be5...,ik1-448-56163,2020-12-29 23:56:47.550595+00:00,181.28.152.133,...,NaN,NaN,NaN,NaN,NaN,var/lib/cowrie/downloads/a8460f446be540410004b...,/root/.ssh/authorized_keys,NaN,NaN,NaN
287560,cowrie.session.file_download,NaN,NaN,a8460f446be540410004b1a8db4083773fa46f7fe76fa8...,1.0,NaN,Saved redir contents with SHA-256 a8460f446be5...,ik1-448-56163,2020-12-29 23:58:40.509255+00:00,51.178.86.97,...,NaN,NaN,NaN,NaN,NaN,var/lib/cowrie/downloads/a8460f446be540410004b...,/root/.ssh/authorized_keys,NaN,NaN,NaN
287682,cowrie.session.file_download,NaN,NaN,a8460f446be540410004b1a8db4083773fa46f7fe76fa8...,1.0,NaN,Saved redir contents with SHA-256 a8460f446be5...,ik1-448-56163,2020-12-29 23:58:54.748012+00:00,201.116.194.210,...,NaN,NaN,NaN,NaN,NaN,var/lib/cowrie/downloads/a8460f446be540410004b...,/root/.ssh/authorized_keys,NaN,NaN,NaN


In [8]:
#ダウンロードに成功
df.query('eventid == "cowrie.session.file_download.failed"')

,eventid,ttylog,size,shasum,duplicate,duration,message,sensor,timestamp,src_ip,...,langCS,username,password,data,id,outfile,destfile,filename,width,height
